# Homework 2-3

3)	Using SKLearn, build an OLS regression model for the Auto MPG Data Set including all explanatory variables (except car name).

a. 	Clean the data, removing samples with empty entries and scaling each feature to have zero mean and unit variance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import statsmodels as sm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/EMSE6575/auto-mpg.csv"
df = pd.read_csv(path)

In [ ]:
df.describe()

,mpg,cylinders,displacement,weight,acceleration,model year,origin
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000


In [ ]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [ ]:
for i in df.columns:
  print(i, "->", len(df[i].unique()))

mpg -> 129
cylinders -> 5
displacement -> 82
horsepower -> 94
weight -> 351
acceleration -> 95
model year -> 13
origin -> 3
car name -> 305


In [ ]:
df = df[df["horsepower"]!= "?"]

In [ ]:
df["horsepower"] = df["horsepower"].astype(str).astype(int)

In [ ]:
df.drop("car name", axis = 1, inplace = True)

In [ ]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scale_df = scaler.fit_transform(df)

df = pd.DataFrame(scale_df, columns = df.columns)
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,-0.698638,1.483947,1.077290,0.664133,0.620540,-1.285258,-1.625315,-0.716641
1,-1.083498,1.483947,1.488732,1.574594,0.843334,-1.466724,-1.625315,-0.716641
2,-0.698638,1.483947,1.182542,1.184397,0.540382,-1.648189,-1.625315,-0.716641
3,-0.955212,1.483947,1.048584,1.184397,0.536845,-1.285258,-1.625315,-0.716641
4,-0.826925,1.483947,1.029447,0.924265,0.555706,-1.829655,-1.625315,-0.716641
...,...,...,...,...,...,...,...,...
387,0.455941,-0.864014,-0.520637,-0.480448,-0.221125,0.021294,1.636410,-0.716641
388,2.636813,-0.864014,-0.932079,-1.364896,-0.999134,3.287676,1.636410,0.526382
389,1.097374,-0.864014,-0.568479,-0.532474,-0.804632,-1.430430,1.636410,-0.716641
390,0.584228,-0.864014,-0.712005,-0.662540,-0.415627,1.110088,1.636410,-0.716641


b. 	What are the best fitting regression coefficients?

In [ ]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

y = df["mpg"]
X = df.drop("mpg", axis = 1)

X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

result = sm.OLS(y_train, X_train).fit()
y_pred = result.predict(X_test)
result.params

const           0.006868
cylinders      -0.075572
displacement    0.202567
horsepower     -0.105052
weight         -0.668380
acceleration    0.013414
model year      0.362352
origin          0.166517
dtype: float64

In [ ]:
print(result.params[abs(result.params) == max(abs(result.params))])

weight   -0.66838
dtype: float64


c. 	How much variance in the data does this model explain?

In [ ]:
from sklearn.metrics import explained_variance_score

In [ ]:
print("Variance: %.2f" % explained_variance_score(y_test, y_pred))

Variance: 0.79


d. 	What is the residual sum of squares?

In [ ]:
import numpy as np
print("Residual Sum of Squares: %.2f" % np.sum(np.square(y_test - y_pred)))

Residual Sum of Squares: 13.93


e. 	Calculate the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) for two models – one with all coefficients, and one with the largest magnitude coefficient. According to each of these criteria, which one of these two is a better model?

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import math
from math import log

In [ ]:
def calculate_aic(n, mse, num_params):
    aic = n * math.log(mse) + 2 * num_params
    return aic

def calculate_bic(n, mse, num_params):
    bic = n * math.log(mse) + num_params * math.log(n)
    return bic

mse = mean_squared_error(y_test, y_pred)
num_params = len(result.params) + 1

In [ ]:
print("AIC coefficients: ", calculate_aic(len(y_test), mse, num_params))
print("BIC coefficients: ", calculate_bic(len(y_test), mse, num_params))

AIC coefficients:  -119.12173416236757
BIC coefficients:  -97.79670349016439


In [ ]:
X_max = X["weight"]
y_max = df['mpg']

X_max_train, X_max_test, y_max_train, y_max_test = train_test_split(X_max, y_max, test_size = 0.2, random_state = 42)

regr_largest = sm.OLS(y_max_train, X_max_train)
result_largest = regr_largest.fit()
y_largest_pred = result_largest.predict(X_max_test)

mse_largest = mean_squared_error(y_max_test, y_largest_pred)
num_params_largest = len(result_largest.params) + 1

In [ ]:
print("Largest coefficient of AIC : ", calculate_aic(len(y_max_test), mse_largest, num_params_largest))
print('Largest coefficient of BIC: ', calculate_bic(len(y_max_test), mse_largest, num_params_largest))

Largest coefficient of AIC :  -95.46070730324416
Largest coefficient of BIC:  -90.72181159831013


AIC, BIC figures illustrates that regression model that use all coefficents gives better results and better model.

f. Using the AIC or BIC criteria, find a better model for this dataset

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import math
from math import log
import itertools

In [ ]:
df2 = df.copy()
df2

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,-0.698638,1.483947,1.077290,0.664133,0.620540,-1.285258,-1.625315,-0.716641
1,-1.083498,1.483947,1.488732,1.574594,0.843334,-1.466724,-1.625315,-0.716641
2,-0.698638,1.483947,1.182542,1.184397,0.540382,-1.648189,-1.625315,-0.716641
3,-0.955212,1.483947,1.048584,1.184397,0.536845,-1.285258,-1.625315,-0.716641
4,-0.826925,1.483947,1.029447,0.924265,0.555706,-1.829655,-1.625315,-0.716641
...,...,...,...,...,...,...,...,...
387,0.455941,-0.864014,-0.520637,-0.480448,-0.221125,0.021294,1.636410,-0.716641
388,2.636813,-0.864014,-0.932079,-1.364896,-0.999134,3.287676,1.636410,0.526382
389,1.097374,-0.864014,-0.568479,-0.532474,-0.804632,-1.430430,1.636410,-0.716641
390,0.584228,-0.864014,-0.712005,-0.662540,-0.415627,1.110088,1.636410,-0.716641


In [ ]:
i = 1
min_aic = np.inf
min_bic = np.inf
best_features = []
best_model = None

y = df2["mpg"]
X = df2.drop("mpg", axis = 1)


def calculate_aic(n, mse, num_params):
    aic = n * math.log(mse) + 2 * num_params
    return aic

def calculate_bic(n, mse, num_params):
    bic = n * math.log(mse) + num_params * math.log(n)
    return bic

n_features = list(range(1,7))
for comb in range(1, 7):
    for combination in itertools.combinations(n_features, comb):
        
        #print(combination)
        X = df2.drop("mpg", axis = 1)
        X = X.iloc[:, (k for k in combination)]
        
        X = sm.add_constant(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        result = sm.OLS(y_train, X_train).fit()
        y_pred = result.predict(X_test)
        #result.params
        
        mse = mean_squared_error(y_test, y_pred)
        num_params = len(result.params) + 1
    
        aic = calculate_aic(len(y_test), mse, num_params)
        bic = calculate_bic(len(y_test), mse, num_params)

        if aic<min_aic or bic<min_bic:
            min_aic = aic
            min_bic = bic
            best_features = X.columns
            

In [ ]:
print("min_aic is : ",min_aic, " and min_bic is : ", min_bic)
print(best_features)

min_aic is :  -130.65269660173874  and min_bic is :  -121.17490519187065
Index(['const', 'weight', 'model year'], dtype='object')


In [ ]:
X = df2[["weight","model year"]]
y = df2["mpg"]

X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

result = sm.OLS(y_train, X_train).fit()
y_pred = result.predict(X_test)
result.params
        
mse = mean_squared_error(y_test, y_pred)
num_params = len(result.params) + 1
    
aic = calculate_aic(len(y_test), mse, num_params)
bic = calculate_bic(len(y_test), mse, num_params)

In [ ]:
print("AIC :",aic," BIC : ",bic)

AIC : -130.65269660173874  BIC :  -121.17490519187065


g. Use 10-fold cross validation and MSE as a metric to select among these three models. How are the results different from using AIC or BIC?

In [ ]:
import sklearn
from sklearn.metrics import fbeta_score, make_scorer
y = df["mpg"]
X_model_1 = df.drop("mpg", axis = 1)
X_model_2 = df["weight"]
X_model_3 = df[["weight", "model year"]]


model = LinearRegression()
mse_1 = cross_val_score(model, X_model_1, y, cv = 10, scoring = make_scorer(sklearn.metrics.mean_squared_error))

mse_2 = cross_val_score(model, X_model_2.values.reshape(-1, 1), y, cv = 10, scoring = make_scorer(sklearn.metrics.mean_squared_error))

mse_3 = cross_val_score(model, X_model_3, y, cv = 10, scoring = make_scorer(sklearn.metrics.mean_squared_error))

In [ ]:
print("MSE 1 : %.3f" %(sum(mse_1) / len(mse_1)))
print("MSE 2 : %.3f" %(sum(mse_2) / len(mse_2)))
print("MSE 3 : %.3f" %(sum(mse_3) / len(mse_3)))

MSE 1 : 0.208
MSE 2 : 0.353
MSE 3 : 0.212
